In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

## Initialize Model

In [2]:
model = ConcreteModel()

## Define Decision Variables

In [3]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
shifts = ['morning', 'evening', 'night']

days_shifts = { day: shifts for day in days }

In [4]:
available_workers = 9 # total workers available which is actually more than needed
workers = ['workers_' + str(i) for i in range(1, available_workers+1)]

In [5]:
# binary variables representing if a worker is scheduled somewhere
model.works = Var(((worker, day, shift) for worker in workers for day in days for shift in days_shifts[day]), 
                    within=Binary, initialize=0)

# binary variables representing if a worker is necessary
model.needed = Var(workers, within=Binary, initialize=0)

# binary variables representing if a worker worked on Sunday but not on Saturday (avoid if possible)
model.no_pref = Var(workers, within=Binary, initialize=0)

## Define Constraints

In [6]:
model.constraints = ConstraintList() # create a set of constraints

In [7]:
# constraint: all shifts are assigned
for day in days:
    for shift in days_shifts[day]:
        if day in days[:-1] and shift in ['morning', 'evening']:
            # weekdays' and saturday's shifts have exactly two workers (except night)
            model.constraints.add(
                2 == sum(model.works[worker, day, shift] for worker in workers)
            )            
        else:
            # sunday's and night's shifts have exactly one worker
            model.constraints.add(
                1 == sum(model.works[worker, day, shift] for worker in workers)
            )

In [8]:
# constraint: no more than 40 hours worked
working_hours = 8

for worker in workers:
    model.constraints.add(
        40 >= sum(working_hours * model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    )

In [9]:
# constraint: rest between two shifts is of 12 hours (i.e., at least two shifts)
for worker in workers:
    for j in range(len(days)):
        # if working in morning, cannot work again on that day
        model.constraints.add(
            1 >= sum(model.works[worker, days[j], shift] for shift in days_shifts[days[j]])
        )
        
        # if working in evening, until next evening (note that after sunday comes next monday)
        model.constraints.add(
            1 >= sum(model.works[worker, days[j], shift] for shift in ['evening', 'night']) + 
                     model.works[worker, days[(j + 1) % 7], 'morning']
        )
        
        # if working in night, until next night
        model.constraints.add(
            1 >= sum(model.works[worker, days[(j + 1) % 7], shift] for shift in ['morning', 'evening']) + 
                     model.works[worker, days[j], 'night']
        )

In [10]:
# constraint: definition of model.needed
for worker in workers:
    model.constraints.add(
        10000 * model.needed[worker] >= sum(model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    )

In [11]:
# constraint: definition of model.no_pref
for worker in workers:
    model.constraints.add(
        model.no_pref[worker] >= sum(model.works[worker, 'Sat', shift] for shift in days_shifts['Sat']) - 
                                 sum(model.works[worker, 'Sun', shift] for shift in days_shifts['Sun'])
    )

## Define Objective Function

In [12]:
def obj_expression(model):
    
    total_workers = len(workers)
    return sum(model.no_pref[worker] for worker in workers) + sum(total_workers * model.needed[worker] for worker in workers)

In [13]:
model.objective_func = Objective(rule=obj_expression, sense=minimize)

## Solve The Model!

In [14]:
opt = SolverFactory('cbc') # download solver in projects.coin-or.org/Cbc to solve a Mixed Integer Programming problem
results = opt.solve(model)

## Report The Result

In [15]:
import json

In [16]:
def get_workers_needed(needed):
    """Extract to a list the needed workers for the optimal solution."""
    workers_needed = []
    for worker in workers:
        if needed[worker].value == 1:
            workers_needed.append(worker)
    return workers_needed

workers_needed = get_workers_needed(model.needed) # dict with the optimal timetable

def get_work_table(works):
    """Build a timetable of the week as a dictionary from the model's optimal solution."""
    week_table = { day: { shift: [] for shift in days_shifts[day] } for day in days }
    for worker in workers:
        for day in days:
            for shift in days_shifts[day]:
                if works[worker, day, shift].value == 1:
                    week_table[day][shift].append(worker)
                    
    return week_table

week_table = get_work_table(model.works) # list with the required workers

def get_no_preference(no_pref):
    """Extract to a list the workers not satisfied with their weekend preference."""
    return [worker for worker in workers if no_pref[worker].value == 1]

workers_no_pref = get_no_preference(model.no_pref) # list with the non-satisfied workers (work on Saturday but not on Sunday)

In [17]:
print('Workers needed:')
for worker in workers_needed:
    print(worker)

Workers needed:
workers_2
workers_3
workers_4
workers_6
workers_7
workers_8
workers_9


In [18]:
print('Work schedule:')
print(json.dumps(week_table, indent=4))

Work schedule:
{
    "Mon": {
        "morning": [
            "workers_3",
            "workers_4"
        ],
        "evening": [
            "workers_8",
            "workers_9"
        ],
        "night": [
            "workers_6"
        ]
    },
    "Tue": {
        "morning": [
            "workers_4",
            "workers_7"
        ],
        "evening": [
            "workers_8",
            "workers_9"
        ],
        "night": [
            "workers_2"
        ]
    },
    "Wed": {
        "morning": [
            "workers_3",
            "workers_7"
        ],
        "evening": [
            "workers_6",
            "workers_8"
        ],
        "night": [
            "workers_2"
        ]
    },
    "Thu": {
        "morning": [
            "workers_3",
            "workers_4"
        ],
        "evening": [
            "workers_6",
            "workers_9"
        ],
        "night": [
            "workers_7"
        ]
    },
    "Fri": {
        "morning": [
         

In [19]:
print('Workers not satisfied by weekend condition:')
for worker in workers_no_pref:
    print(worker)

Workers not satisfied by weekend condition:
workers_3
workers_4


In [20]:
print('The optimal objective value:', model.objective_func())

The optimal objective value: 65.0


---